In [ ]:
import pickle as pkl
from google.colab import files
upoads=files.upload()

Saving image_list.pkl to image_list.pkl


In [ ]:
uploads2=files.upload()

Saving refined_dataset.xlsx to refined_dataset.xlsx


In [ ]:
import pandas as pd
import numpy as np
df=pd.read_excel('refined_dataset.xlsx')

In [ ]:
df_new=pd.DataFrame()

In [ ]:
img_path=open("image_list.pkl",'rb')
img_list=pkl.load(img_path)
print(img_list[0])

[[[193 133  79]
  [198 147  97]
  [193 138  87]
  ...
  [206 183 157]
  [205 182 156]
  [203 190 182]]

 [[195 137  86]
  [195 138  87]
  [202 145  93]
  ...
  [211 183 159]
  [204 180 154]
  [205 183 172]]

 [[198 141  89]
  [194 137  86]
  [197 140  89]
  ...
  [197 168 141]
  [204 181 155]
  [204 181 165]]

 ...

 [[123 116 107]
  [100  94  79]
  [112 108  89]
  ...
  [ 11  41  36]
  [ 16  64  51]
  [ 12  50  42]]

 [[109 102  85]
  [108  99  89]
  [ 98  93  81]
  ...
  [  3  53  41]
  [ 19  70  60]
  [ 43 126 112]]

 [[ 98  89  82]
  [ 93  85  76]
  [ 83  79  74]
  ...
  [ 53 125 185]
  [ 39 112 184]
  [ 58 131 209]]]


In [ ]:
df_new['img']=img_list
df_new['cat']=df['categoryName']

In [ ]:
from sklearn.model_selection import train_test_split
train_set,test=train_test_split(df_new,test_size=0.2)
train_x=train_set['img']
one_hot_encoded_data = pd.get_dummies(train_set, columns = ['cat'])
train_y=one_hot_encoded_data.drop(columns=['img'])


In [ ]:
print(train_x)

2340    [[[146, 159, 226], [216, 172, 139], [229, 175,...
660     [[[30, 94, 65], [35, 67, 83], [121, 144, 152],...
2199    [[[243, 243, 243], [244, 241, 226], [242, 235,...
2761    [[[91, 81, 151], [85, 81, 152], [87, 80, 153],...
2166    [[[237, 238, 236], [238, 239, 237], [241, 242,...
                              ...                        
120     [[[248, 243, 240], [247, 242, 239], [249, 245,...
1988    [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...
2117    [[[242, 217, 201], [241, 217, 199], [242, 217,...
1540    [[[254, 254, 254], [254, 254, 254], [254, 254,...
1906    [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...
Name: img, Length: 2628, dtype: object


In [ ]:
import tensorflow
import keras
from keras import layers,Model,Input
from keras.models import Sequential

In [ ]:
def make_model():
  inp=Input(shape=(28,28,3))
  conv1=layers.Conv2D(32,(3,3),activation='selu',kernel_initializer='he_uniform')(inp)
  pool1=layers.MaxPool2D(2,2)(conv1)
  conv2=layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform')(pool1)
  pool2=layers.MaxPool2D(2,2)(conv2)
  flt=layers.Flatten()(pool2)
  mlp1=layers.Dense(256,activation='linear',name='dense1')(flt)
  output=layers.Dense(11,activation='softmax')(mlp1)
  model=Model(inputs=inp,outputs=output)
  return model


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator()
#train_x_new=datagen.fit(train_x)

In [ ]:
print(train_x[0].shape)

(28, 28, 3)


In [ ]:
train_x_new=np.array(train_x.tolist()).reshape((2628,28,28,3))
#for img in train_x:
  #img_arr=np.array(img).reshape((28,28,3))
  #train_x_new.append(img_arr)


In [ ]:
datagen.fit(train_x_new)

In [ ]:
len(train_x_new)

2628

In [ ]:
train_y_new=list()
for y in train_y:
  train_y_new

In [ ]:
mdl=make_model()
mdl.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
mdl.fit(train_x_new,train_y,epochs=10,batch_size=10)

Epoch 1/10
263/263 [==============================] - 4s 11ms/step - loss: 50.5619 - accuracy: 0.3497
Epoch 2/10
263/263 [==============================] - 4s 14ms/step - loss: 1.8452 - accuracy: 0.4193
Epoch 3/10
263/263 [==============================] - 3s 11ms/step - loss: 1.7305 - accuracy: 0.4269
Epoch 4/10
263/263 [==============================] - 3s 11ms/step - loss: 1.6849 - accuracy: 0.4346
Epoch 5/10
263/263 [==============================] - 3s 11ms/step - loss: 1.6701 - accuracy: 0.4418
Epoch 6/10
263/263 [==============================] - 4s 13ms/step - loss: 1.6547 - accuracy: 0.4441
Epoch 7/10
263/263 [==============================] - 3s 11ms/step - loss: 1.6482 - accuracy: 0.4486
Epoch 8/10
263/263 [==============================] - 3s 11ms/step - loss: 1.7621 - accuracy: 0.4395
Epoch 9/10
263/263 [==============================] - 3s 12ms/step - loss: 1.6789 - accuracy: 0.4357
Epoch 10/10
263/263 [==============================] - 3s 12ms/step - loss: 1.6586 - accur

In [ ]:
intermediate_layer_model = Model(inputs=mdl.input,outputs=mdl.get_layer('dense1').output)
intermediate_output = intermediate_layer_model.predict(train_x_new)

83/83 [==============================] - 1s 8ms/step


In [ ]:
test_x=test['img']
print(test_x.shape)

(658,)


In [ ]:
test_x_new=np.array(test_x.tolist()).reshape((658,28,28,3))
test_y=pd.get_dummies(test[''])

In [ ]:
y_pred=mdl.predict(test_x_new)

21/21 [==============================] - 0s 8ms/step


In [ ]:
print(y_pred)

[[0.41040412 0.01155563 0.00265556 ... 0.17569484 0.02156099 0.06109858]
 [0.41040412 0.01155563 0.00265556 ... 0.17569484 0.02156099 0.06109858]
 [0.41040412 0.01155563 0.00265556 ... 0.17569484 0.02156099 0.06109858]
 ...
 [0.41040412 0.01155563 0.00265556 ... 0.17569484 0.02156099 0.06109858]
 [0.41040415 0.01155564 0.00265556 ... 0.17569485 0.02156099 0.06109858]
 [0.4517805  0.00995978 0.00217029 ... 0.14859946 0.02337495 0.04820997]]
